# Singletons - Dependently Typed Programming

In [2]:
:ext DataKinds
:ext RankNTypes
:ext GADTs
:ext KindSignatures
:ext LambdaCase
:ext TypeApplications

data DoorState = Open | Closed | Locked
  deriving (Show, Eq)

data Door (s :: DoorState) = UnsafeMakeDoor { doorMaterial :: String }

Here we are using a phantom type to make the state of a door a type level parameter. This allows us to write functions that can only receive or return a door with a specific state.

We can do the same thing using a GADT

In [3]:
data Door :: DoorState -> * where
  UnsafeMakeDoor :: { doorMaterial :: String } -> Door s

To be able to inspect the DoorState on the value level or create a door with a state that is unknown until runtime, we need singletons

In [6]:
data SingDS :: DoorState -> * where
  SOpen :: SingDS 'Open
  SClosed :: SingDS 'Closed
  SLocked :: SingDS 'Locked
  
doorState :: SingDS s -> Door s -> DoorState
doorState SOpen _ = Open
doorState SClosed _ = Closed
doorState SLocked _ = Locked

By passing in the singleton along with the door, ensuring that the type parameters match, we can effectively pattern match on types.

We can use a type class to have the singleton passed implicitly:

In [11]:
class SingDSI s where
  singDS :: SingDS s
  
instance SingDSI 'Open where
  singDS = SOpen
instance SingDSI 'Closed where
  singDS = SClosed
instance SingDSI 'Locked where
  singDS = SLocked
  
doorState_ :: SingDSI s => Door s -> DoorState
doorState_ = doorState singDS

The compiler deduces which singleton to pass in based on the type parameter of Door, which is an instance of the class.

We have turned a function where the SingDS was passed explicitly into one where it is passed implicitly. To do the reverse (implicit to explicit) we need a utility function:

In [13]:
withSingDSI :: SingDS s -> (SingDSI s => r) -> r
withSingDSI sng x = case sng of
  SOpen -> x
  SClosed -> x
  SLocked -> x
  
doorStateExp s d = withSingDSI s (doorState_ d)

Using singletons to write a nice door constructor:

In [15]:
mkDoor :: SingDS s -> String -> Door s
mkDoor _ = UnsafeMakeDoor

:t mkDoor SOpen "Oak"

mkDoor SOpen "Oak" :: Door 'Open